# 6 - Transformers for Sentiment Analysis

In this notebook we will be using the transformer model, first introduced in [this](https://arxiv.org/abs/1706.03762) paper. Specifically, we will be using the BERT (Bidirectional Encoder Representations from Transformers) model from [this](https://arxiv.org/abs/1810.04805) paper. 

Transformer models are considerably larger than anything else covered in these tutorials. As such we are going to use the [transformers library](https://github.com/huggingface/transformers) to get pre-trained transformers and use them as our embedding layers. We will freeze (not train) the transformer and only train the remainder of the model which learns from the representations produced by the transformer. In this case we will be using a multi-layer bi-directional GRU, however any model can learn from these representations.

In [0]:
#all imports
import tensorflow as tf
import torch

In [0]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


## Preparing Data

First, as always, let's set the random seeds for deterministic results.

In [0]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla P100-PCIE-16GB


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!unzip -P ****** -qq '/content/drive/My Drive/GreekData/Greek.zip'

replace Greek/offenseval-greek-training-v1.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Greek/readme-trainingset-greek.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
import torch

import random
import numpy as np

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
!pip install transformers

The transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

Luckily, the transformers library has tokenizers for each of the transformer models provided. In this case we are using the BERT model which ignores casing (i.e. will lower case every word). We get this by loading the pre-trained `bert-base-uncased` tokenizer.

In [0]:
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)

The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [0]:
len(tokenizer.vocab)

119547

Using the tokenizer is as simple as calling `tokenizer.tokenize` on a string. This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.

In [0]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hell', '##o', 'world', 'how', 'are', 'you', '?']


We can numericalize tokens using our vocabulary using `tokenizer.convert_tokens_to_ids`.

In [0]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[61694, 10133, 11356, 14796, 10301, 13028, 136]


The transformer was also trained with special tokens to mark the beginning and end of the sentence, detailed [here](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel). As well as a standard padding and unknown token. We can also get these from the tokenizer.

**Note**: the tokenizer does have a beginning of sequence and end of sequence attributes (`bos_token` and `eos_token`) but these are not set and should not be used for this transformer.

In [0]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


We can get the indexes of the special tokens by converting them using the vocabulary...

In [0]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


...or by explicitly getting them from the tokenizer.

In [0]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

In [0]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']

print(max_input_length)

512


Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

In [0]:
def tokenize_and_cut(sentence):
    '''if z not in string.punctuation  and t.is_stop==False:
                x2.append(z)
                   
    sentence1=' '.join((x2))  '''  
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [0]:

          
def preprocess1(arrt):
    doc=[]
    emojis={}
    #noises = ['@USER','n\'t', '\'s', '\'m',"’"]
    allTokens =[]
    for txt in arrt:
        k=0
        sentVect=[]
        txt=p2.clean(txt)
        txt=p1.sub("",txt)
        x1=nlp(txt)
        x2=[]
        for t in x1:
            z=str(t)
            if z not in string.punctuation  and t.is_stop==False:
                x2.append(z)
                   
        
        sentence=' '.join((x2))         
        allTokens.append(sentence)      
    return allTokens


Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.

We define the label field as before.

In [0]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [0]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Greek/offenseval-greek-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]


In [0]:
greektrain,greektest= train_test_split(greekdata, test_size=0.2, random_state=42)
export_csv = greektrain.to_csv ('Greek/TrainFilegreek.csv', index = None, header=True)
print (greektrain.head())
export_csv = greektest.to_csv ('Greek/TestFilegreek.csv', index = None, header=True)
print (greektest.head())

        id                                              tweet subtask_a
2002  4347  @USER Μη με κουρδίζεις νυχτιάτικα, βαλτός είσα...       NOT
4645  4175                         Ωραιο μουνι, ποδοσφαιρικο.       OFF
1916  1439  @USER 1. Αποκλείστηκες διότι με έβαλες σε ομάδ...       NOT
1550  6404                 Κόψτε τους τα μανιτάρια #XFactorGR       NOT
2358  5730  Σακης... Χριστινακι μου γλυκο! στα καπακια εισ...       NOT
        id                                              tweet subtask_a
4538  2263  @USER Mπλοκ το Μαράκι, δεν της άρεσε η λύση πο...       NOT
7353  1055  @USER αδελφέ, δεν ξέρω τι ακριβώς έγινε.. απλά...       NOT
3181  7142  Τα γαλλικά, ήταν η γλώσσα της Αριστοκρατίας τό...       NOT
1626  3768  @USER Ρε Κουλη σε ξέρουμε επειδή φαίνεσαι και ...       NOT
7634  5698  Από το πρωί είμαι με ένα «τι κοιτάς ρε μαλακισ...       OFF


We load the data and create the validation splits as before.

In [0]:
from torchtext import datasets
from torchtext import data

train_val_fields = [
    ('id', None), # we dont need this, so no processing
    ('tweet', TEXT), # process it as label
    ('subtask_a', LABEL), # we dont need this, so no processing
]
train_data, valid_data = data.TabularDataset.splits(path='Greek/', 
                                            format='csv', 
                                            train='TrainFilegreek.csv', 
                                            validation='TestFilegreek.csv', 
                                            fields=train_val_fields, 
                                            skip_header=True)


In [0]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")

Number of training examples: 6995
Number of validation examples: 1749


We can check an example and ensure that the text has already been numericalized.

In [0]:
print(vars(train_data.examples[6]))

{'tweet': [476, 13517, 62152, 10484, 466, 97317, 33617, 31796, 13140, 70887, 43202, 31821, 465, 31625, 10649, 483, 29223, 14669, 35670, 45878, 27393, 465, 24767, 107, 471, 480, 16146, 31712, 18015, 469, 17198, 30645, 64472, 27621, 108, 22013, 70279, 53166, 487, 13140, 63444, 61399, 61399, 107, 108, 22013, 70279, 53166, 168, 28174, 12961, 38278], 'subtask_a': 'NOT'}


We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

In [0]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['tweet'])

print(tokens)

['μ', '##ο', '##λι', '##ς', 'β', '##γη', '##κε', 'αν', '##α', '##κο', '##ιν', '##ωση', 'α', '##πο', 'την', 'σ', '##π', '##υ', '##ρι', '##δου', '##λα', 'α', '##κα', '"', 'η', 'π', '##ρ', '##ω', '##τη', 'ε', '##λ', '##λη', '##νι', '##δα', '#', 'master', '##chef', '##gr', 'χ', '##α', '##χ', '##χα', '##χα', '"', '#', 'master', '##chef', '##gr', '_', 'xe', '##ft', '##iles']


Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

In [0]:
LABEL.build_vocab(train_data)

In [0]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f9a84579d90>, {'NOT': 0, 'OFF': 1})


As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

In [0]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE, 
    sort=False,
    device = device)

## Build the Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [0]:
from transformers import BertTokenizer, BertModel
from transformers import BertConfig



In [0]:
config1=BertConfig.from_pretrained('bert-base-multilingual-cased',output_hidden_states=True)


In [0]:
bert =BertModel.from_pretrained('bert-base-multilingual-cased',config=config1)


Next, we'll define our actual model. 

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialization is standard.

Within the forward pass, we wrap the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. The transformer actually returns the embeddings for the whole sequence as well as a *pooled* output. The [documentation](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel) states that the pooled output is "usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence", hence we will not be using it. The rest of the forward pass is the standard implementation of a recurrent model, where we take the hidden state over the final time-step, and pass it through a linear layer to get our predictions.

In [0]:
import torch.nn as nn
from transformers import BertConfig
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']*4
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded= self.bert(text)
        hidden=embedded[-1]
        #print(hidden[12].size())
        #print(embedded[0].size())
        #print(len(hidden))
        '''
        encoded_layers=hidden
        print ("Number of layers:", len(encoded_layers))
        layer_i = 0

        print ("Number of batches:", len(encoded_layers[layer_i]))
        batch_i = 0

        print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
        token_i = 0

        print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))
        '''

        token_embeddings=torch.stack(hidden, dim=0)
        #print(token_embeddings.size())
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        #print(token_embeddings.size())
        token_embeddings = token_embeddings.permute(1,2,0,3)
        #print(token_embeddings.size())
        token_vecs_cat = []
        irnn=[]
        for b in token_embeddings:
          token_vecs_cat = []
          for token in b:
           cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
           #print("cat_vec",len(cat_vec))
           token_vecs_cat.append(cat_vec)
           #a=torch.Tensor(token_vecs_cat)
           #print(len(token_vecs_cat))
          a = torch.stack(token_vecs_cat)#.to(device)
          irnn.append(a) 
        z=torch.stack(irnn)#.to(device)
        '''print(z.size())
        print ('Shape is: %d x %d x %d' % (len(z), len(z[0]),len(z[1])))
        print(type(z))
        print(type(z[0]))
        print(type(z[1]))
        '''
        _, hidden = self.rnn(z)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [0]:
import torch
a=[[[1,2,3],[2,3,4]],[[1,2,3],[2,3,4]]]
b=torch.Tensor(a)
print(b)
print(b.size())

tensor([[[1., 2., 3.],
         [2., 3., 4.]],

        [[1., 2., 3.],
         [2., 3., 4.]]])
torch.Size([2, 2, 3])


Next, we create an instance of our model using standard hyperparameters.

In [0]:
HIDDEN_DIM = 512
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 193,595,393 trainable parameters


In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

In [0]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,741,953 trainable parameters


We can double check the names of the trainable parameters, ensuring they make sense. As we can see, they are all the parameters of the GRU (`rnn`) and the linear layer (`out`).

In [0]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## Train the Model

As is standard, we define our optimizer and criterion (loss function).

In [0]:
import torch.optim as optim
from transformers import AdamW

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

Place the model and criterion onto the GPU (if available)

In [0]:
model = model.to(device)
criterion = criterion.to(device)

Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def binary_accuracy2(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    #print("Predicted",rounded_preds)
    #print("labelsFlattend",y)
    return f1_score(y.detach().cpu().numpy(), rounded_preds.detach().cpu().numpy(), average='macro'),acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        #print(batch)
        optimizer.zero_grad()
        
        predictions = model(batch.tweet).squeeze(1)
        
        loss = criterion(predictions, batch.subtask_a)
        
        acc = binary_accuracy(predictions, batch.subtask_a)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    nb_eval_steps=0
    eval_acc=0
    eval_f1=0
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.tweet).squeeze(1)
            
            loss = criterion(predictions, batch.subtask_a)
            
            tmpf1score,acc = binary_accuracy2(predictions, batch.subtask_a)
            eval_f1 = eval_f1+tmpf1score
            eval_acc=eval_acc+acc
            nb_eval_steps += 1
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
 
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    nb_eval_steps=0
    eval_acc=0
    eval_f1=0
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.tweet).squeeze(1)
            
            loss = criterion(predictions, batch.subtask_a)
            
            tmpf1score,acc = binary_accuracy2(predictions, batch.subtask_a)
            eval_f1 = eval_f1+tmpf1score
            eval_acc=eval_acc+acc
            nb_eval_steps += 1
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
 
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we'll train our model. This takes considerably longer than any of the previous models due to the size of the transformer. Even though we are not training any of the transformer's parameters we still need to pass the data through the model which takes a considerable amount of time on a standard GPU.

In [0]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
    #    torch.save(model.state_dict(), '/content/drive/My Drive/GreekData/bertgreekgru.pth.tar')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

torch.Size([128, 144, 768])
torch.Size([128, 157, 768])
torch.Size([128, 155, 768])
torch.Size([128, 158, 768])
torch.Size([128, 148, 768])
torch.Size([128, 139, 768])
torch.Size([128, 152, 768])
torch.Size([128, 136, 768])
torch.Size([128, 143, 768])
torch.Size([128, 149, 768])
torch.Size([128, 152, 768])
torch.Size([128, 150, 768])
torch.Size([128, 168, 768])
torch.Size([128, 153, 768])
torch.Size([128, 166, 768])
torch.Size([128, 151, 768])
torch.Size([128, 160, 768])
torch.Size([128, 162, 768])
torch.Size([128, 142, 768])
torch.Size([128, 147, 768])
torch.Size([128, 141, 768])
torch.Size([128, 143, 768])
torch.Size([128, 148, 768])
torch.Size([128, 150, 768])
torch.Size([83, 145, 768])
torch.Size([128, 147, 768])
torch.Size([128, 150, 768])
torch.Size([128, 139, 768])
torch.Size([128, 141, 768])
torch.Size([128, 151, 768])
torch.Size([128, 167, 768])
torch.Size([128, 144, 768])
torch.Size([128, 143, 768])
torch.Size([128, 141, 768])
torch.Size([128, 149, 768])
torch.Size([128, 152,

We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [0]:
torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreekgru.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreekgru.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

## Inference

We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [0]:
predict_sentiment(model, tokenizer, "This film is terrible")

In [0]:
predict_sentiment(model, tokenizer, "This film is great")